In [ ]:
from IPython.display import Image  # handy, builtin capability to make stuff look good

# our standard imports
import numpy as np
import pandas as pd

# Functions

---

**class-derived example of a monte carlo simulation**

**Sections:**

0. Intro
1. Random Number Generation
2. Building a Turn
3. Looping a Turn
4. Creating and Visualizing a Results Matrix
5. Increased Iterations

---

### Intro:

This example is taken from an assingment withing the intro to BI class at UGA taught by Dr. Hugh Watson.

Reference: *Watson, H.J., and J.H. Blackstone, Computer Simulation, 2nd edition, (New York: Wiley, 1989)*

The premise of this problem is inventory simulation with regards to two "random" variables (demand, delivery time) and four cost variables (carrying, ordering, and stockout). 

It's presumed that we either know the probabilities associated with the random variables, or have enough information to make a good guess. To Watson's point, "Despite the feelings of discomfort most people have with subjective probabilities, decisions must be made, and carefully reasoned subjective estimates are better than no estimates at all." Based on these probabilities, random number tables are references to determine levels for our random variables. 

The probability distributions:

In [ ]:
Image(filename='random_numbers.png') 

With these distributions, and a series of rules, we can build a "turn" (each line of an order/delivery timeline) that is iterable.

In each turn there's a chance that we will have to order, recieve an order, or just wait. Regardless of actions taken, there are always costs to calculate. Below is the example of this exercise done by hand for 20 iterations. From this, we can derive all of our rules.

In [ ]:
Image(filename='monte_carlo.png')

---

### Random Number Generation:

The crux of this exercise is actually the easiest part. It's also the easiest place to make a mistake, but it's the easiest mistake to fix. Easiest.

The package numpy offers *probably* all of the rng capabilities you'll ever need: random normal, binomial, poisson, uniform, beta, gamma, dirchlet, etc...

In [ ]:
# random is the default, uniform between 0 and 1

np.random.random()

In [ ]:
# running it again produces a new random number

np.random.random()

In [ ]:
# we need a replicable experiment though, so we need to set a "seed"

np.random.seed(42)
np.random.random(3)  # number of numbers

In [ ]:
# again

np.random.seed(42)
np.random.random(3)

In [ ]:
# perfect, now we can generate all of our numbers
# we'll start at 20 numbers, like the referenced text

np.random.seed(42)

random_numbers = np.random.random(20)

In [ ]:
# using the tables from earlier, we can make functions to generate
# our demand and delivery times based on random_numbers

# functions go like: 
# def function_name(variables):
#    '''explanation of your function'''
#    stuff stuff stuff
#    return result

def get_demand(order_count):
    '''
    get_demand(order_count)
    ---
    order_count = current number of orders made
    returns: amount of demand based on random_numbers[index][2:4]
    if random_number[index] = 0.37454012, then i = 37
    '''
    
    i = random_numbers[order_count]
    i = str(i)[2:4]  # convert the random number to a string and index
    i = int(i)  # and convert back
    
    if i < 10:  # 0 - 09
        demand = 0
    elif i < 30:  # 10 - 29
        demand = 1
    elif i < 60:  # 30 = 59
        demand = 2
    elif i < 84:  # 60 - 83
        demand = 3
    else:  # 84 - 99
        demand = 4
        
    return demand

In [ ]:
# try it out, any indices from 0 - 19 since we made 20 observations

get_demand()

In [ ]:
# delivery is shorter, but exactly the same idea
# reorder_count moves much slower than order_count,
# so it will not be a limiting factor

def get_delivery_time(reorder_count):
    '''
    get_delivery_time(reorder_count)
    ---
    reorder_count = current number of reorders made
    returns: time until delivery based on random_numbers[index][4:6]
    if random_number[index] = 0.37454012, then i = 45
    '''
    
    i = random_numbers[reorder_count]
    i = str(i)[4:6]  # convert the random number to a string and index
    i = int(i)  # and convert back
    
    if i < 40:  # 0 - 39
        time = 1
    elif i < 80:  # 40 - 79
        time = 2
    else:  # 80 - 99
        time = 3
        
    return time

In [ ]:
# try it out, any indices from 0 - 19 since we made 20 observations

get_delivery_time()

If you're wondering, "what's the point of that, did we really have to?"

Well, no, we didn't have to, but let's say we needed to use get_demand 10 times in a task later on. That's no fun, and it's also the point of functions: they abstract tedious work that has to be done - usually done multiple times - into the background so we can make a good-looking, streamlined process.

---

### Building a Turn

Looking at the simulation chart, let's go through all of the steps chronologically and then get a full turn.

**steps:**
1. if order outstanding and time == 0 then order recieved (add to inventory)
2. get demand
3. subtract demand from inventory
    1. if inventory > demand, no worries
    2. else sell as much as possible before determining stockout quantity (20 each)
4. check if it's time to reorder
    1. get delivery time
    2. add reorder cost (30 per order)
5. add cost of holding (2 per item)
6. decrease order time

In [ ]:
# there are a few things we need to keep track of along the way:
# counts: order and reorder
# set variables: order quantity and reorder point
# time to next order, and if we have an order outstanding
# inventory balance, and cost incurred in the turn

def turn(order_count, reorder_count,
         order_quantity, reorder_point,
         order_outstanding, order_time,
         balance, cost):
    '''
    using all variables, execute one run of the simulation
    work(variables)
    ---
    see above for variable descriptions
    '''

    # ordering rule
    if order_outstanding == 1 and order_time == 0:
        balance += order_quantity
        order_outstanding = 0  # allows for a new order
       
    # selling stuff
    demand = get_demand(order_count)
    
    # stock in
    if balance >= demand:
        balance -= demand
    # stock out
    else:
        # give partial credit
        if balance > 0:
            demand -= balance
            balance = 0  # all gone
        # incur stockout cost on remaining demand
        cost += 20 * demand
    
    # reordering rule
    if balance <= reorder_point and order_outstanding == 0:  # order_outstanding prevents multiple orders
        order_time = get_delivery_time(reorder_count)
        reorder_count += 1
        order_outstanding = 1
        cost += 30
    
    # cost of holding
    cost += 2 * balance
    
    # time is constant
    if order_time > 0:
        order_time -= 1
        
    # after the turn, check all of our variables
    return (reorder_count,
            order_outstanding, order_time,
            balance, cost)

---

### Looping a Turn

Now that we have a single turn down, we need to figure out a way to take multiple turns.

In [ ]:
# most of the variables per turn are internal.
# all we have to worry about specifying before we start are:
# order_quantity: the amount to order
# reorder_point: the amount to order at/below
# balance: the amount we start with (== order_quantity in the text)
# turns: the number of turns we want to take


def monte_crisco(order_quantity, reorder_point, balance, turns):
    '''
    monte_crisco(order_quantity, reorder_point, balance, turns)
    ----
    order_quantity: if reordering, order "order_quantity"
    reorder_point: reorder if balance <= "reorder_point"
    balance: starting balance
    turns: number of turn() executions
    '''
    
    # starting point, no cost or counts
    total_cost = 0  # we'll add each turn's cost here
    # order_count increments every turn, so we can use the turn number
    reorder_count = 0
    order_outstanding = 0
    order_time = 0
    
    # we can use a while loop, and keep track of runs vs some integer i (turn number)
    i = 0
    while i < turns:
        # turn execution
        turn_data = turn(reorder_count = reorder_count,
                         order_quantity = order_quantity, 
                         reorder_point = reorder_point,
                         order_outstanding = order_outstanding, 
                         order_time = order_time,
                         balance = balance, 
                         cost = 0,  # cost is on a per turn basis
                         order_count = i)  # again, order_count is simply the turn number
#         print(turn_data)
        # increment cost
        total_cost += turn_data[-1]
        # update all other variables
        reorder_count = turn_data[0]
        order_outstanding = turn_data[1]
        order_time = turn_data[2]
        balance = turn_data[3]
        # next turn!
        i += 1
        
    # the text asks for average cost, we can remove "/runs" and get total cost
    return total_cost/turns

In [ ]:
# and now take a turn, any quantity and order point you want

order_quantity = 
reorder_point = 
num_turns = 20

monte_crisco(order_quantity, reorder_point, order_quantity, num_turns)

---

### Creating and Visualizing a Results Matrix

Alright, now we need to compare different values for order_quantity and reorder_point across the same number of turns and same random numbers.

Let's draw from notebook two to create a dataframe for holding the data from our runs, and then hop into matplotlib to graph our results.

In [ ]:
%%time

# %%time is a magical function to tell you the time it takes to run a cell

# store info for all runs in a matrix
matrix = []

for q in range(4, 11):  # order_quantities 4 - 10
    row = []  # I'm assuming that quantities are row and reorders a column, it's trivial to switch
    for r in range(1, 8):  # reorder_point 1 - 7
        row.append(monte_crisco(q, r, q, 20)) 
    matrix.append(row)  # after each row is finished, start the next
    
# matrix -> dataframe
pd.DataFrame(data=matrix, index=range(4, 11), columns=range(1, 8))

In [ ]:
# you COULD stop here and throw those numbers at someone
# or you could make it look good

# data for plotting
plt_data = []
# just running through the same data
for q in range(4, 11):
    row = []
    for r in range(1, 8):
        z = monte_crisco(q, r, q, 20)
        plt_data.append((q, r, z))  # save q, r as x, y and avg cost as height (z)
    matrix.append(row)

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt  # standard import
from mpl_toolkits.mplot3d import Axes3D  # special 3D graphing

# let's get fancy and find the minimum point
minimum_idx = min([(val[2], idx) for idx, val in enumerate(plt_data)])[1]
print(plt_data[minimum_idx])  # show
# conditinoal colors, r = red and k = black
colors = ['r' if idx == minimum_idx else 'k' for idx in range(len(plt_data))]

In [ ]:
# time to plot

fig = plt.figure(figsize=(10, 10))  # set the output size
ax = Axes3D(fig)  # specify 3D plotting
ax.scatter([i[0] for i in plt_data], # x
           [i[1] for i in plt_data], # y
           [i[2] for i in plt_data], # z
           c=colors) # colors
plt.show()

---

### Increased Iterations

That plot is kinda wonky. Let's smooth it out by sending the number of iterations towards (emphasis on the -wards, but if you want to sit around for-literally-ever go ahead and don't mind me) infinity.

All we have to do is make more random numbers and change the last variable in monte_crisco() to be higher... that's it.

In [ ]:
# set replicable list of 100k random numbers
np.random.seed(42)
random_numbers = np.random.random(100000)

In [ ]:
%%time

# patience, this takes a few seconds
# just copy pasted from above

# data for plotting
plt_data = []
# matrix of convergences
matrix = []
for q in range(4, 11):
    row = []
    for r in range(1, 8):
        z = monte_crisco(q, r, q, len(random_numbers))
        row.append(z)
        plt_data.append((q, r, z))
    matrix.append(row)
    
# we don't even really need this, but if you wanted a "deliverable" you could 
# do convergences.to_csv('path.csv') and send that off
# convergences = pd.DataFrame(data=matrix, index=range(4, 11), columns=range(1, 8))

In [ ]:
# copy paste copy paste copy paste

minimum_idx = min([(val[2], idx) for idx, val in enumerate(plt_data)])[1]
colors = ['r' if idx == minimum_idx else 'k' for idx in range(len(plt_data))]

fig = plt.figure(figsize=(10, 10))  # set the output size
ax = Axes3D(fig)  # specify 3D plotting
ax.scatter([i[0] for i in plt_data], # x
           [i[1] for i in plt_data], # y
           [i[2] for i in plt_data], # z
           c=colors) # colors
plt.show()  # freaking gorgeous now